In [80]:
from google.cloud import bigquery as gbq
import pandas as pd

In [81]:
client = gbq.Client(project = 'wedge-project0')

In [82]:
def query_owners(project_id,dataset_id,sample_size):
    query =  f"""
        with owner_sample as(

        select distinct card_no 
        from `{project_id}.{dataset_id}.transArchive_*`
        order by rand()
        limit {sample_size}
        )
        select * 
        from owner_sample;

        """
    query_job = client.query(query,project = project_id)
    query_df = query_job.result()
    query_df = query_df.to_dataframe()
    
    return query_df

In [83]:
def card_no_records(project_id,dataset_id,query_df):

    owner_list = tuple(query_df['card_no'])

    query = f""" 
    select * 
    from `{project_id}.{dataset_id}.transArchive_*`
    where card_no in {owner_list}
    """

    query_job = client.query(query,project = project_id)
    query_result = query_job.result()
    query_result = query_result.to_dataframe()

    return query_result

In [84]:
data = query_owners('wedge-project-0','transactions',440)

card_no_records = card_no_records('wedge-project-0','transactions',data)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [85]:
card_no_records.to_csv('card_no_records_sample.txt',sep = '\t', index = False)